# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tralee,52.2704,-9.7026,11.99,93,6,1.48,IE,1693633377
1,1,battle ground,45.7809,-122.5334,21.61,82,17,1.34,US,1693633186
2,2,blackmans bay,-43.0167,147.3167,14.83,70,51,2.24,AU,1693633377
3,3,bethel,41.3712,-73.4140,15.36,86,0,0.00,US,1693633377
4,4,roxas,11.5853,122.7511,32.30,64,100,2.39,PH,1693633378


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:


# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"  
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
best_weather_df = city_data_df.loc[(city_data_df["Humidity"] < 75) | (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
city_udata_df = best_weather_df.dropna()


# Display sample data
city_udata_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tralee,52.2704,-9.7026,11.99,93,6,1.48,IE,1693633377
1,1,battle ground,45.7809,-122.5334,21.61,82,17,1.34,US,1693633186
2,2,blackmans bay,-43.0167,147.3167,14.83,70,51,2.24,AU,1693633377
3,3,bethel,41.3712,-73.4140,15.36,86,0,0.00,US,1693633377
4,4,roxas,11.5853,122.7511,32.30,64,100,2.39,PH,1693633378


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_udata_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tralee,IE,52.2704,-9.7026,93,
1,battle ground,US,45.7809,-122.5334,82,
2,blackmans bay,AU,-43.0167,147.3167,70,
3,bethel,US,41.3712,-73.4140,86,
4,roxas,PH,11.5853,122.7511,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
limit = 20


# set up a parameters dictionary
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

#Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]

    latitude = row["Lat"]
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tralee - nearest hotel: The Ashe Hotel
battle ground - nearest hotel: Best Western Plus Battle Ground Inn & Suites
blackmans bay - nearest hotel: Villa Howden
bethel - nearest hotel: Hampton Inn Danbury
roxas - nearest hotel: Roxas President's Inn
cam ranh - nearest hotel: Khách sạn Thái Bình
nemuro - nearest hotel: イーストハーバーホテル
severo-yeniseyskiy - nearest hotel: Актолик
bilibino - nearest hotel: No hotel found
vadso - nearest hotel: Scandic Vadsø
jamestown - nearest hotel: DoubleTree Jamestown
ekibastuz - nearest hotel: General
waitangi - nearest hotel: Hotel Chathams
kununurra - nearest hotel: Kununurra Country Club Resort
port-aux-francais - nearest hotel: Keravel
utrik - nearest hotel: No hotel found
usinsk - nearest hotel: общежитие Лукойл-Коми
port lincoln - nearest hotel: Boston Hotel
juneau - nearest hotel: Ramada by Wyndham Juneau
bubaque - nearest hotel: Chez Julio
puerto natales - nearest hotel: Dorotea Patagonia Hostel
hermanus - nearest hotel: Aloe gu

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tralee,IE,52.2704,-9.7026,93,The Ashe Hotel
1,battle ground,US,45.7809,-122.5334,82,Best Western Plus Battle Ground Inn & Suites
2,blackmans bay,AU,-43.0167,147.3167,70,Villa Howden
3,bethel,US,41.3712,-73.4140,86,Hampton Inn Danbury
4,roxas,PH,11.5853,122.7511,64,Roxas President's Inn
...,...,...,...,...,...,...
563,dadukou,CN,26.5479,101.7054,60,锦江之星酒店
564,timmins,CA,48.4669,-81.3331,72,The Senator Hotel & Conference Centre
565,tukrah,LY,32.5341,20.5791,69,No hotel found
566,raja,SS,8.4596,25.6780,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:

#Configure the map plot
country_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
country_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)